In [52]:
import os
import re
import ollama
import numpy as np
from pypdf import PdfReader
from typing import *

In [20]:
pdf_path = '../.docs/venus_and_adonis.pdf'

In [21]:
reader = PdfReader(pdf_path)

In [23]:
pages = [page.extract_text(extraction_mode='layout') for page in reader.pages]
raw_text = ''.join(pages)

In [25]:
print(raw_text)

Venus and Adonis                                                                                                                                                           https://shakespeare.mit.edu/Poetry/VenusAndAdonis.html

                  Venus and Adonis

                                'Vilia miretur vulgus; mihi flavus Apollo
                                Pocula Castalia plena ministret aqua.'

                                TO THE
                                RIGHT HONORABLE HENRY WRIOTHESLY,
                                EARL OF SOUTHAMPTON, AND BARON OF TICHFIELD.
                                RIGHT HONORABLE,

                  I KNOW not how I shall offend in dedicating my unpolished lines to your lordship, nor how the world will
                  censure me for choosing so strong a prop to support so weak a burden only, if your honour seem but pleased,
                  I account myself highly praised, and vow to take advantage of all idle hours, till I have ho

In [38]:
chunks = raw_text.split(sep='\n')
chunks = [chunk.strip() for chunk in chunks]
chunks = [chunk for chunk in chunks if chunk not in ['']]
chunks

['Venus and Adonis                                                                                                                                                           https://shakespeare.mit.edu/Poetry/VenusAndAdonis.html',
 'Venus and Adonis',
 "'Vilia miretur vulgus; mihi flavus Apollo",
 "Pocula Castalia plena ministret aqua.'",
 'TO THE',
 'RIGHT HONORABLE HENRY WRIOTHESLY,',
 'EARL OF SOUTHAMPTON, AND BARON OF TICHFIELD.',
 'RIGHT HONORABLE,',
 'I KNOW not how I shall offend in dedicating my unpolished lines to your lordship, nor how the world will',
 'censure me for choosing so strong a prop to support so weak a burden only, if your honour seem but pleased,',
 'I account myself highly praised, and vow to take advantage of all idle hours, till I have honoured you with',
 'some graver labour. But if the first heir of my invention prove deformed, I shall be sorry it had so noble a',
 'god-father, and never after ear so barren a land, for fear it yield me still so bad a harvest

In [43]:
def normalize_spaces(text: str) -> str:
    return re.sub(r'\s+', ' ', text).strip()

In [45]:
def remove_urls(text: str) -> str:
    text = re.sub(r'http\S+|www\.\S+', '', text)
    return re.sub(r'\s+', ' ', text).strip()

In [46]:
chunks = [normalize_spaces(chunk) for chunk in chunks]
chunks = [remove_urls(chunk) for chunk in chunks]
chunks

['Venus and Adonis',
 'Venus and Adonis',
 "'Vilia miretur vulgus; mihi flavus Apollo",
 "Pocula Castalia plena ministret aqua.'",
 'TO THE',
 'RIGHT HONORABLE HENRY WRIOTHESLY,',
 'EARL OF SOUTHAMPTON, AND BARON OF TICHFIELD.',
 'RIGHT HONORABLE,',
 'I KNOW not how I shall offend in dedicating my unpolished lines to your lordship, nor how the world will',
 'censure me for choosing so strong a prop to support so weak a burden only, if your honour seem but pleased,',
 'I account myself highly praised, and vow to take advantage of all idle hours, till I have honoured you with',
 'some graver labour. But if the first heir of my invention prove deformed, I shall be sorry it had so noble a',
 'god-father, and never after ear so barren a land, for fear it yield me still so bad a harvest. I leave it to your',
 "honourable survey, and your honour to your heart's content; which I wish may always answer your own wish",
 "and the world's hopeful expectation.",
 "Your honour's in all duty,",
 'WIL

In [47]:
def normalize_chunks_lengths(chunks: List[str], minimum_chunk_length: int = 150, join_str: str = ' '):
    normalized_chunks = ['']
        
    for chunk in chunks:
        if len(normalized_chunks[-1]) < minimum_chunk_length:
            normalized_chunks[-1] = normalized_chunks[-1] + join_str + chunk
        else:
            normalized_chunks.append(chunk)

    return normalized_chunks

In [48]:
normalized_chunks = normalize_chunks_lengths(chunks)
normalized_chunks

[" Venus and Adonis Venus and Adonis 'Vilia miretur vulgus; mihi flavus Apollo Pocula Castalia plena ministret aqua.' TO THE RIGHT HONORABLE HENRY WRIOTHESLY,",
 'EARL OF SOUTHAMPTON, AND BARON OF TICHFIELD. RIGHT HONORABLE, I KNOW not how I shall offend in dedicating my unpolished lines to your lordship, nor how the world will',
 'censure me for choosing so strong a prop to support so weak a burden only, if your honour seem but pleased, I account myself highly praised, and vow to take advantage of all idle hours, till I have honoured you with',
 'some graver labour. But if the first heir of my invention prove deformed, I shall be sorry it had so noble a god-father, and never after ear so barren a land, for fear it yield me still so bad a harvest. I leave it to your',
 "honourable survey, and your honour to your heart's content; which I wish may always answer your own wish and the world's hopeful expectation. Your honour's in all duty,",
 "WILLIAM SHAKESPEARE. EVEN as the sun with purp

In [53]:
embedding_model_name = 'mxbai-embed-large'
ollama.pull(embedding_model_name)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [93]:
def embed_chunks(chunks: List[str]) -> np.typing.ArrayLike:
    embs = [ollama.embed(model=embedding_model_name, input=input_str) for input_str in chunks]
    embs = [emb.get('embeddings')[0] for emb in embs]
    embs = np.array(embs, dtype=np.float32)
    return embs

In [94]:
embed_chunks([
    'I like cats',
    'The feline specie felis catus is the best',
    'We\'re the cat lovers society!'
])


array([[-0.01551637,  0.0364796 ,  0.02681714, ..., -0.00502469,
        -0.00743345, -0.04733381],
       [ 0.00287699,  0.02242767,  0.03456862, ..., -0.0043806 ,
         0.0054952 , -0.025636  ],
       [-0.02030589,  0.06353579, -0.00642451, ..., -0.00572764,
        -0.03525757, -0.05633152]], dtype=float32)

In [95]:
chunk_embeds = embed_chunks(chunks)
chunk_embeds.shape, chunk_embeds

((1241, 1024),
 array([[-0.00323791, -0.00884696,  0.04918234, ...,  0.02223267,
          0.01374046, -0.01734457],
        [-0.00323791, -0.00884696,  0.04918234, ...,  0.02223267,
          0.01374046, -0.01734457],
        [ 0.00160071,  0.01458935,  0.04492112, ...,  0.00508998,
         -0.00263359, -0.03548667],
        ...,
        [ 0.04016061, -0.0110938 , -0.01158206, ..., -0.07369687,
          0.04398278, -0.01872427],
        [-0.02553174, -0.03070035, -0.02675067, ..., -0.01039915,
         -0.00059525,  0.01218947],
        [ 0.02565262,  0.01636529, -0.01544681, ..., -0.03337607,
         -0.06753365, -0.02882367]], dtype=float32))

In [117]:
def retrieve(input_str: str, chunks: List[str], chunk_embeddings: np.typing.ArrayLike, top_k: int) -> List[str]:
    input_embs = ollama.embed(model=embedding_model_name, input=input_str)
    input_embs = np.array(input_embs.get('embeddings')[0], dtype=np.float32)
    scores = np.linalg.matmul(chunk_embeddings, input_embs)
    top_k_indexes = np.argpartition(scores, -top_k)[-top_k:]
    top_k_indexes = np.sort(top_k_indexes)[::-1]
    relevant_chunks = [chunks[index] for index in top_k_indexes]
    
    return relevant_chunks


retrieve('How brave is the lion?', chunks, chunk_embeds, 10)

['Put fear to valour, courage to the coward.',
 "'To see his face the lion walk'd along",
 'But the blunt boar, rough bear, or lion proud,',
 'Being ireful, on the lion he will venture:',
 'And careless lust stirs up a desperate courage,',
 'Who is so faint, that dare not be so bold',
 'Thin mane, thick tail, broad buttock, tender hide:',
 'With gentle majesty and modest pride;',
 'Imperiously he leaps, he neighs, he bounds,',
 'Courageously to pluck him from his horse.']

In [120]:
generative_model_name = 'gemma3:4b'

In [121]:
ollama.pull(generative_model_name)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [162]:
def parse_prompt(context: str, relevant_chunks: List[str], question: str) -> str:
    context_prompt = f'CONTEXT\n{context}\n\n'
    text_chunks_prompt = f'ORIGINAL TEXT CHUNKS\n{'\n'.join(relevant_chunks)}\n\n'
    question_prompt = f'QUESTION\n{question}'
    prompt = context_prompt + text_chunks_prompt + question_prompt
    return prompt

In [172]:
def pretty_response(prompt: str, answer: str):
    print(f'-----------PROMPT-----------')
    print(prompt)
    print(f'-----------PROMPT-----------\n')
    
    print(f'-----------ANSWER-----------')
    print(answer)
    print(f'-----------ANSWER-----------')

In [177]:
def ask_llm(prompt: str) -> str:
    return ollama.generate(model=generative_model_name, prompt=prompt).get('response')

In [181]:
def rag(question: str, context: str = None) -> str:
    relevant_chunks = retrieve(input_str=question, chunks=chunks, chunk_embeddings=chunk_embeds, top_k=10)
    if not context: context = 'You\'re an expert in Shakespeare. We\'re studying his classical poem Venus and Adonis'
    prompt = parse_prompt(
        context=context,
        relevant_chunks=relevant_chunks,
        question=question
    )
    answer = ask_llm(prompt)
    pretty_response(prompt, answer)

In [183]:
ask_llm('How is Venus interpreted in the poem Venus and Adonis?')

"In Edmund Spenser's *Venus and Adonis*, Venus is a remarkably complex and layered figure, far more than just a simple romantic rejection. She's not simply a jealous wife, but a multi-faceted being representing a potent blend of **primal instinct, maternal love, religious devotion, and even a chillingly detached, almost regal, power.** Here's a breakdown of how she’s interpreted throughout the poem:\n\n**1. The Primordial Force of Desire & Instinct:**\n\n* **Initially, she’s presented as the embodiment of raw, untamed desire.** The opening images of her lounging, bathing, and admiring Adonis evoke a sensual, almost animalistic energy. She’s driven by a basic, fundamental need for connection and pleasure. Spenser emphasizes her control over her own body and her awareness of Adonis's physical beauty.\n* **Her bathing scenes are key.** They highlight her immediate, instinctive reaction to his attractiveness, portraying her as a powerful, captivating force.\n\n**2. The Mother Figure & Mate

In [175]:
rag('How is Venus interpreted in the poem?')

-----------PROMPT-----------
CONTEXT
You're an expert in Shakespeare. We're studying his classical poem Venus and Adonis

ORIGINAL TEXT CHUNKS
This solemn sympathy poor Venus noteth;
Venus salutes him with this fair good-morrow:
So glides he in the night from Venus' eye.
Open'd their mouths to swallow Venus' liking.
'Ay me,' quoth Venus, 'young, and so unkind?
So he were like him and by Venus' side.
Nature that made thee, with herself at strife,
Sick-thoughted Venus makes amain unto him,
Venus and Adonis
Venus and Adonis

QUESTION
How is Venus interpreted in the poem?
-----------PROMPT-----------

-----------ANSWER-----------
Ah, a wonderfully perceptive question! Let’s delve into the complex and layered interpretation of Venus within *Venus and Adonis*. It’s crucial to understand that Shakespeare isn’t presenting a simple, straightforward depiction of love – he's constructing a powerful, almost unsettling, image of desire and its consequences. 

Essentially, Venus in this poem is port

In [176]:
rag('what is the purpose of the owl in the poem?')

-----------PROMPT-----------
CONTEXT
You're an expert in Shakespeare. We're studying his classical poem Venus and Adonis

ORIGINAL TEXT CHUNKS
'Grim-grinning ghost, earth's worm, what dost thou mean
She hearkens for his hounds and for his horn:
Lo, here the gentle lark, weary of rest,
'Now of this dark night I perceive the reason:
The owl, night's herald, shrieks, ''Tis very late;'
Thy eyes' shrewd tutor, that hard heart of thine,
For through his mane and tail the high wind sings,
Anon he starts at stirring of a feather;
'Bid me discourse, I will enchant thine ear,
Nature that made thee, with herself at strife,

QUESTION
what is the purpose of the owl in the poem?
-----------PROMPT-----------

-----------ANSWER-----------
Ah, an excellent question! The inclusion of the owl in Shakespeare’s *Venus and Adonis* is a remarkably subtle and layered one, and one that’s often overlooked in a hurried reading. It’s far more than just a rustic sound effect. 

The owl, you see, functions as a key 

In [ ]:
ask_llm('what is the purpose of the owl in the poem Venus and Adonis?')

"The owl in Edmund Spenser's *Venus and Adonis* is a profoundly complex and layered symbol, serving multiple, often contradictory purposes. It's arguably the most significant and debated symbol in the entire poem. Here's a breakdown of its multifaceted roles:\n\n**1. Harbinger of Doom & Mortality:**\n\n* **The Most Obvious Interpretation:** Initially, and most prominently, the owl embodies death and impending doom. It’s repeatedly associated with Adonis’s wound and his approaching death. The relentless hooting of the owl symbolizes the inevitability of mortality and the harsh reality of the natural world. This is particularly evident in the opening stanzas where the owl's cries repeatedly announce Adonis’s suffering.\n* **Echoes of Classical Mythology:** This connection to death aligns with classical associations of owls, particularly in Greek and Roman mythology, where they were linked to Hades, the god of the underworld.\n\n**2. A Symbol of Venus’s Jealousy & Fury:**\n\n* **Venus's A

In [185]:
answer_rag = """Ah, an excellent question! The inclusion of the owl in Shakespeare’s *Venus and Adonis* is a remarkably subtle and layered one, and one that’s often overlooked in a hurried reading. It’s far more than just a rustic sound effect. 

The owl, you see, functions as a key symbolic device, primarily representing **mortality, decay, and the intrusion of the darker, more unsettling aspects of nature into the idealized world of love and beauty that Venus embodies.** 
...
So, in short, the owl isn’t merely a sound effect. It’s a deliberate, powerfully symbolic device used to underscore the poem's themes of mortality, conflict, and the precariousness of beauty and love. 

Do you find this interpretation compelling? Would you like to delve into how Shakespeare uses other elements – like the wind – to reinforce this central idea?"""

answer_pure_llm = """Ah, a wonderfully perceptive question! Let’s delve into the complex and layered interpretation of Venus within *Venus and Adonis*. It’s crucial to understand that Shakespeare isn’t presenting a simple, straightforward depiction of love – he's constructing a powerful, almost unsettling, image of desire and its consequences. 

Essentially, Venus in this poem is portrayed as a *ravenous, almost predatory* force. She’s not merely a gentle, romantic figure. The text drips with this unsettling quality. Let’s break down how it’s achieved:

1.  **“Glides he in the night from Venus’ eye. Open’d their mouths to swallow Venus’ liking.”** – This is absolutely key. The imagery of “gliding” and “open’d their mouths” immediately evokes a sense of being drawn, consumed, and perhaps even violated. It’s not a loving pursuit, but a forceful taking. She doesn't *seek* Adonis, she *captures* him. 

2.  **“Sick-thoughted Venus”** – This phrase is vital. "Sick-thoughted" suggests a consuming, unhealthy obsession. It implies a feverish, almost pathological desire. She’s not acting out of affection, but from a tormented, almost violent impulse.

3.  **“So he were like him and by Venus’ side.”** - This reinforces the idea that her desire is not born of admiration, but of a need to *transform* Adonis into something that perfectly satisfies her tormented fancy. She wants to mold him, dominate him. 

4. **Her "sympathy" is clearly manipulative.** She uses phrases like "Ay me," not with genuine sorrow for Adonis’ fate, but to encourage and sustain her own obsession.

**Therefore, in essence, Shakespeare presents Venus as an embodiment of uncontrolled, primal desire – a force of nature that both creates and threatens to destroy.** She’s not a character to be pitied; she's a figure of immense, terrifying power. 

It’s important to remember that this interpretation aligns with the broader context of Shakespeare’s exploration of lust and its dangers. He’s not glorifying love; he's warning against the seductive allure of unchecked passion."""

ask_llm(f'We\'re studying the poem Venus and Adonis, from Shakespeare. We asked the following question to two LLMs A and B: \'what is the purpose of the owl in the poem Venus and Adonis?\'. Bellow, there are the two answers. Which one is the best?\n\nANSWER FROM LLM A:\n{answer_pure_llm}\n\nANSWER FROM LLM B:{answer_rag}')

'LLM B’s answer is the significantly better response. Here\'s a breakdown of why:\n\n*   **Directly Addresses the Question:** LLM B immediately answers the core question – what *is* the owl’s purpose? It doesn\'t start with a broader interpretation of Venus as a “ravenous, almost predatory” force (as LLM A does).\n*   **Clear Symbolism:** LLM B provides a concise and accurate interpretation – the owl represents "mortality, decay, and the intrusion of the darker, more unsettling aspects of nature." This is the central, most defensible interpretation of the owl’s role in the poem.\n*   **Focus and Precision:** The response is tightly focused on the owl\'s symbolic function.\n*   **Conciseness:** It\'s more direct and avoids overly elaborate or tangential explanations.\n\n**Why LLM A’s answer is weaker:**\n\n*   **Digression:** LLM A spends a lot of time discussing Venus\'s overall portrayal and then tries to force the owl into that framework. This is unnecessary and distracting.\n*   **L